In [1]:
import numpy as np
import scipy as sp
from skimage import io
from matplotlib import pyplot as plt
from skimage import transform
from tqdm import tqdm
from time import time
import pickle
import os
import pandas as pd
from glob import glob
import mrcfile
from skimage import registration as im_reg_met
%matplotlib inline

from utils import shift, iou, polar_trfm
from registration import (
    set_integration_intervals,
    laguerre_zeros_precompute, 
    image_fbt_precompute, 
    fbm_registration,
    laguerre_functions_precompute
)

In [2]:
from matrix_utils import *
import cv2

def normalize(im):
    return (im - im.min()) / (im.max() - im.min())

def shift(im, vec):
    mat_trans = get_translation_mat(*vec)
    return cv2.warpAffine(im, get_mat_2x3(mat_trans), 
                          (im.shape[1], im.shape[0]))
    
    
def rotate(im, angle, center=None):
    h, w = im.shape[:2]
    if center is None:
        center = np.array([w // 2, h // 2])
    mat_trans_minus_center = get_translation_mat(-center[0], -center[1])
    mat_rot = get_rotation_mat(angle, radians=False)
    mat_trans_center = get_translation_mat(*center)
    return cv2.warpAffine(im, get_mat_2x3(mat_trans_center @ mat_rot @ mat_trans_minus_center),
                          (im.shape[1], im.shape[0]))


def apply_transform(image, transform_dict, center = None):
    h, w = image.shape[:2]
    if center is None:
        center = np.array([w // 2, h // 2])
        
    ksi = transform_dict['ksi']
    etta_prime = transform_dict['etta']
    omegga_prime = transform_dict['omegga']
    etta = etta_prime - ksi
    omegga = omegga_prime - etta_prime
    eps = transform_dict['eps']
    com_offset = transform_dict['com_offset']

    rho = (2* com_offset**2 * (1 + np.cos(etta + eps))) ** 0.5
    tx, ty = rho * np.cos(ksi), rho * np.sin(ksi)
    alpha = etta + eps + omegga + ksi
#     print('alpha', np.degrees(alpha))

    im_reg = rotate(image, -np.degrees(alpha))

    im_reg = shift(im_reg, (-tx, -ty))
    
    return im_reg, [np.degrees(alpha), tx, ty]


def save_arr_mrc(fname, arr):
    with mrcfile.new(fname, overwrite=True) as mrc:
        mrc.set_data(np.array(arr))
    


In [3]:
def run_fast_fbm_laguerre(seq, func_parameters):
    df = pd.DataFrame(columns=['x', 'y', 'ang', 'ksi', 'eta_prime', 'omega_prime', 'dft_x', 'dft_y'])
    fbm_seq = [seq[0].copy()]
    fbm_seq_shift = [seq[0].copy()]
    
    fixed_image = normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3))
    func_parameters = fixed_image_precompute(fixed_image,
                                             func_parameters)
    
    for i in tqdm(range(1, len(seq))):
        reg1 = fbm_registration(fixed_image,
                                normalize(sp.ndimage.gaussian_filter(seq[i].copy(), 1.3)),
                                image_radius=image_radius, p_s=pixel_sampling, com_offset=com_offset_initial,
                                method='fast_fbm_laguerre', lag_func_num=lag_func_num, lag_scale=lag_scale,
                                masks=None, shift_by_mask=False, additional_params=func_parameters)
        
        im_reg1, params = apply_transform(seq[i].copy(), reg1, center)
        fbm_seq.append(im_reg1.copy())

#         print('IoU:', iou(seq[0], im_reg1))
        shifts = im_reg_met.phase_cross_correlation(normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3)),
                                                    normalize(sp.ndimage.gaussian_filter(im_reg1.copy(), 1.3)),
                                                    upsample_factor=100)[0][::-1]

        fbm_seq_shift.append(shift(im_reg1, -shifts))
        founded_values = [params[1], params[2], params[0], reg1['ksi'],
                          reg1['etta'], reg1['omegga'], shifts[0], shifts[1]]
        df.loc[i-1] = founded_values
        
    fbm_seq = np.array(fbm_seq)
    return df, fbm_seq, fbm_seq_shift


def run_fbm(seq, func_parameters):
    df = pd.DataFrame(columns=['x', 'y', 'ang', 'ksi', 'eta_prime', 'omega_prime', 'dft_x', 'dft_y'])
    fbm_seq = [seq[0].copy()]
    fbm_seq_shift = [seq[0].copy()]
    fixed_image = normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3))
    func_parameters = fixed_image_precompute(fixed_image,
                                             func_parameters)
    for i in tqdm(range(1, len(seq))):
        reg1 = fbm_registration(fixed_image,
                                normalize(sp.ndimage.gaussian_filter(seq[i].copy(), 1.3)),
                                image_radius=image_radius, p_s=pixel_sampling, com_offset=com_offset_initial,
                                method='fbm', masks=None, shift_by_mask=False,
                                additional_params=func_parameters)
    #                             method='fbm_laguerre', lag_func_num=lag_func_num, lag_scale=lag_scale,
    #                             masks=None, shift_by_mask=False, additional_params=params)
        im_reg1, params = apply_transform(seq[i].copy(), reg1, center)
        fbm_seq.append(im_reg1.copy())

#         print('IoU:', iou(seq[0], im_reg1))
        shifts = im_reg_met.phase_cross_correlation(normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3)),
                                                    normalize(sp.ndimage.gaussian_filter(im_reg1.copy(), 1.3)),
                                                    upsample_factor=100)[0][::-1]

        fbm_seq_shift.append(shift(im_reg1, -shifts))
        founded_values = [params[1], params[2], params[0], reg1['ksi'],
                          reg1['etta'], reg1['omegga'], shifts[0], shifts[1]]
        df.loc[i-1] = founded_values
        
    fbm_seq = np.array(fbm_seq)
    return df, fbm_seq, fbm_seq_shift


def run_fbm_laguerre(seq, func_parameters):
    df = pd.DataFrame(columns=['x', 'y', 'ang', 'ksi', 'eta_prime', 'omega_prime', 'dft_x', 'dft_y'])
    fbm_seq = [seq[0].copy()]
    fbm_seq_shift = [seq[0].copy()]
    fixed_image = normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3))
    func_parameters = fixed_image_precompute(fixed_image,
                                             func_parameters)
    for i in tqdm(range(1, len(seq))):
        reg1 = fbm_registration(fixed_image,
                                normalize(sp.ndimage.gaussian_filter(seq[i].copy(), 1.3)),
                                image_radius=image_radius, p_s=pixel_sampling, com_offset=com_offset_initial,
                                method='fbm_laguerre', masks=None, shift_by_mask=False,
                                lag_func_num=lag_func_num, lag_scale=lag_scale, 
                                additional_params=func_parameters)
        im_reg1, params = apply_transform(seq[i].copy(), reg1, center)
        fbm_seq.append(im_reg1.copy())

#         print('IoU:', iou(seq[0], im_reg1))
        shifts = im_reg_met.phase_cross_correlation(fixed_image,
                                                    normalize(sp.ndimage.gaussian_filter(im_reg1.copy(), 1.3)),
                                                    upsample_factor=100)[0][::-1]

        fbm_seq_shift.append(shift(im_reg1, -shifts))
        founded_values = [params[1], params[2], params[0], reg1['ksi'],
                          reg1['etta'], reg1['omegga'], shifts[0], shifts[1]]
        df.loc[i-1] = founded_values
        
    fbm_seq = np.array(fbm_seq)
    return df, fbm_seq, fbm_seq_shift


def fixed_image_precompute(image, additional_params):
    if additional_params is None:
        additional_params = {}
        
    if 'integration_intervals' in additional_params:
        Im1, Ih1, Imm, theta_net, \
        u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth = additional_params['integration_intervals']
    else:
        Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth  = \
            set_integration_intervals(image_radius, p_s, com_offset)
        additional_params['integration_intervals'] = \
        Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth 
    

    if 'polar_fixed' in additional_params:
        pol1 = additional_params['polar_fixed']
    else:
        maxrad = image_radius
        pol1 = polar_trfm(image, int(2 * bandwidth), int(2 * bandwidth / np.pi), maxrad)

    if 'precomputed_fbt_fixed' in additional_params:
        Fm_arr = additional_params['precomputed_fbt_fixed']
    else:
        alphas = []
        for it_m1 in range(len(Im1)):
            m1 = Im1[it_m1]
            for it_h1 in range(len(Ih1)):
                h1 = Ih1[it_h1]
                for it_mm in range(len(Imm)):
                    mm = Imm[it_mm]
                    if m1 + h1 + mm in alphas:
                        continue
                    alphas.append(m1 + h1 + mm)
        # print('Fm precompute')
        if method in ['fbm_laguerre', 'fast_fbm_laguerre'] and \
                'laguerre_functions' not in additional_params:
            additional_params['laguerre_functions'] = laguerre_functions_precompute(alphas, x_net,
                                                                                   lag_func_num, lag_scale)
        Fm_arr = image_fbt_precompute(pol1, alphas, theta_net, u_net, x_net,
                                      method, lag_func_num, lag_scale,
                                      lag_num_dots,
                                      additional_params)
        additional_params['precomputed_fbt_fixed'] = Fm_arr
    return additional_params

In [4]:
from pathlib import Path

global_path = '/home/ubuntu/Data/cryo-em samples/synthetic_data_2023'
folder = 'small_trans'
model = 'model2096'
name = '_stack.mrc'
params_name = '_info.csv'
num = 2


path = Path(global_path) / folder / model / f'{num}{name}'

In [5]:
results_path = Path(global_path) / 'results'
os.makedirs(str(results_path), exist_ok=True)

method = 'fast_fbm_laguerre'
method_path = results_path / method
os.makedirs(str(method_path), exist_ok=True)

In [6]:
from skimage import registration as im_reg_met

image_radius = 100
pixel_sampling = 1.
com_offset_initial = 8
lag_func_num=50
lag_scale=5
lag_num_dots = 1000
center = 128, 128

In [9]:
def precompute():
    Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth = \
            set_integration_intervals(image_radius, pixel_sampling, com_offset_initial,verbose=True)
    alphas = []
    for it_m1 in tqdm(range(len(Im1))):
        m1 = Im1[it_m1]
        for it_h1 in range(len(Ih1)):
            h1 = Ih1[it_h1]
            for it_mm in range(len(Imm)):
                mm = Imm[it_mm]
                if abs(m1 + h1 + mm) in alphas:
                    continue
                alphas.append(abs(m1 + h1 + mm))

    params = {}
    if os.path.exists(f'cryo_laguerre_zeros_{len(alphas)}_{lag_func_num}.pkl'):
        with open(f'cryo_laguerre_zeros_{len(alphas)}_{lag_func_num}.pkl', 'rb') as file:
            params['lag_zeros'] = pickle.load(file)  
    else:
        params['lag_zeros'] = laguerre_zeros_precompute(alphas, lag_func_num+1, abort_after=0.1)
        with open(f'cryo_laguerre_zeros_{len(alphas)}_{lag_func_num}.pkl', 'wb') as file:
            pickle.dump(params['lag_zeros'], file)
    params['integration_intervals'] = [Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, 
                                   psi_net, eta_net, eps, b, bandwidth]

    laguerre_functions = laguerre_functions_precompute(alphas, x_net, lag_func_num, lag_scale)
    params['laguerre_functions'] = laguerre_functions

    c1_coefs = np.zeros((len(Im1), len(x_net)))
    for it_m1 in range(len(Im1)):
        m1 = Im1[it_m1]
        c1 = sp.special.jv(m1, b * x_net) * x_net
        c1_coefs[it_m1, :] = c1
    params['precomputed_c1_coefs'] = c1_coefs

    c2_coefs = np.zeros((len(Ih1), len(x_net)))
    for it_m1 in range(len(Im1)):
        for it_h1 in range(len(Ih1)):
            h1 = Ih1[it_h1]
            if it_m1 == 0:
                c2 = sp.special.jv(h1, b * x_net)
                c2_coefs[it_h1, :] = c2
    params['precomputed_c2_coefs'] = c2_coefs
    return params

In [10]:
params = precompute()

for method, func in [('fbm', run_fbm), ('fbm_laguerre', run_fbm_laguerre), 
                     ('fast_fbm_laguerre', run_fast_fbm_laguerre)]:
    
    method_path = results_path / method
    os.makedirs(str(method_path), exist_ok=True)

    opath = method_path / folder / model
    os.makedirs(opath, exist_ok=True)
    opath = method_path / folder / model / str(path).split('/')[-1]
    if not os.path.exists(opath):
        with mrcfile.open(path) as mrc:
            seq = mrc.data
        df, fbm_seq, fbm_seq_shift = func(seq, params)
        save_arr_mrc(opath, fbm_seq)
        df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))

    for snr_value in [0.1, 0.5, 1., 2.]:
        data_name  = str(Path(global_path) / folder / model / f'{snr_value}/{num}{name}')
        
        opath = method_path / folder / model / str(snr_value)
        os.makedirs(str(opath), exist_ok=True)
        opath = method_path / folder / model / str(snr_value) / data_name.split('/')[-1]
        
        if not os.path.exists(opath):
            with mrcfile.open(data_name) as mrc:
                seq = mrc.data
            df, fbm_seq, fbm_seq_shift = func(seq, params)
        
            save_arr_mrc(opath, fbm_seq)
            df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))


100
s_ang = 314.1592653589793
s_rad = 100.0
bandwidth = 157.07963267948966
len(Im1) 25 len(Ih1) 9 len(Imm) 158


  2%|▉                                           | 2/99 [00:06<05:36,  3.47s/it]


KeyboardInterrupt: 

In [ ]:
with mrcfile.open(path) as mrc:
        seq = mrc.data

In [ ]:
method = 'fast_fbm_laguerre'

functions = {'fbm': run_fbm, 'fbm_laguerre': run_fbm_laguerre, 'fast_fbm_laguerre': run_fast_fbm_laguerre}

func = functions[method]

In [ ]:
from skimage import registration as im_reg_met

image_radius = 120
pixel_sampling = 0.5
com_offset_initial = 10
lag_func_num=100
lag_scale=5
lag_num_dots = 2000
center = 128, 128

In [ ]:
params = precompute()

method_path = results_path / method
os.makedirs(str(method_path), exist_ok=True)

opath = method_path / folder / model
os.makedirs(opath, exist_ok=True)
opath = method_path / folder / model / str(path).split('/')[-1]

with mrcfile.open(path) as mrc:
    seq = mrc.data
df, fbm_seq, fbm_seq_shift = func(seq, params)
save_arr_mrc(opath, fbm_seq)
df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))

# for snr_value in [0.1, 0.5, 1., 2.]:
#     for data_name in glob(str(Path(global_path) / folder / model / 
#                               f'{snr_value}/{num}{name}')):
#         with mrcfile.open(data_name) as mrc:
#             seq = mrc.data
#         df, fbm_seq, fbm_seq_shift = func(seq, params)
#         opath = method_path / folder / model / str(snr_value)
#         os.makedirs(str(opath), exist_ok=True)
#         opath = method_path / folder / model / str(snr_value) / data_name.split('/')[-1]
#         save_arr_mrc(opath, fbm_seq)
#         df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))
